In [ ]:
%pip install fastbook transformers tokenizers==0.12.0

CHAP 3

In [ ]:
%pip install --upgrade --force-reinstall protobuf==3.20

In [ ]:
import fastbook
import torch
from fastai.vision.all import *
from fastbook import *
import numpy as np
import numpy.random as nprnd

fastbook.setup_book()



In [ ]:
def quad(a,b,c,x): return a*x**2 + b*x + c
r = quad(3,2,1,1.5)
print(r)

def make_quad(a,b,c): return lambda x: a*x**2 + b*x + c
q = make_quad(3,2,1)
print(q(1.5))
np.random.seed(42)
def noise(x,scale):  return nprnd.normal(scale=scale,size=x.shape) 
def add_noise(x,mult,add): return x + mult*noise(x,add)

x = torch.linspace(-2,2,steps=20)[:,None]
y = add_noise(q(x),0.3,1.5)
plt.scatter(x,y)


In [ ]:
from ipywidgets import interact
""" @interact(a=1.5,b=1.5,c=1.5) """
def plot_quad(a,b,c):
    f = make_quad(a,b,c)
    loss= mse(f(x),y)
    plot_function(f,title=f'Loss: {loss}')
    plt.scatter(x,y)

def mse(preds,acts): return ((preds-acts)**2).mean().sqrt()
def quad_mse(params):
    f= make_quad(*params)
    return mse(f(x),y)

def rectified_linear(m,b,x):
    y = m*x+b
    return torch.clip(y,0,)
def double_relu(m1,b1,m2,b2,x):
    return rectified_linear(m1,b1,x) + rectified_linear(m2,b2,x)
abc = torch.tensor((1.5,1.5,1.5),requires_grad=True)
loss = quad_mse(abc)
print(f'Loss: {loss}')
loss.backward()

abc.grad
with torch.no_grad():
    abc -= abc.grad * 0.01
    loss = quad_mse(abc)

print(f'Loss: {loss}')

for i in range(20):
    loss = quad_mse(abc)
    loss.backward()
    with torch.no_grad():
        abc -= abc.grad * 0.01
        abc.grad.zero_()
    """ print(f'Loss: {loss}') """

    

In [ ]:
@interact(m1=1.5,b1=1.5,m2=1.5,b2=1.5)
def plot_rdouble_relu(m1,b1,m2,b2):
    plot_function(partial(double_relu,m1,b1,m2,b2))
    plt.scatter(x,y)
# 3x3 matrix
matrix1 = torch.tensor([[1, 2, 1],
                        [0, 1, 0],
                        [2, 3, 4]])

# 3x2 matrix
matrix2 = torch.tensor([[2, 5],
                        [6, 7],
                        [1, 8]])
result = torch.matmul(matrix1, matrix2)
result

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.nn import Linear
import torch.optim as optim
import torch
import torch.nn as nn

class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    
df = pd.read_csv('./train.csv')
scaler = MinMaxScaler()
columns_to_normalize = ['Age', 'Fare','SibSp','Parch']
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
df_encoded = pd.get_dummies(df,columns=['Sex','Embarked','Pclass'])
df_encoded.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
# Convert boolean columns to integers
bool_columns = df_encoded.select_dtypes(include='bool').columns
df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(df_encoded, df['Survived'], test_size=0.2)

# Convert DataFrames to PyTorch Tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

model = SimpleNeuralNetwork(input_size=df_encoded.shape[1], hidden_size=16, output_size=1)
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(200):
    y_pred = model(X_train_tensor)

    loss = torch.mean((y_pred.squeeze() - y_train_tensor) ** 2)

    loss.backward()

    optimizer.step()

y_pred = model(X_test_tensor)
accuracy = ((y_pred > 0.5).squeeze().float() == y_test_tensor).float().mean()
# Run the model on the entire test dataset
y_pred_test = model(X_test_tensor)

# Convert the predictions to binary labels using a threshold of 0.5
y_pred_binary = (y_pred_test > 0.5).squeeze().float()

# Convert the PyTorch tensor containing the predictions to a NumPy array
y_pred_numpy = y_pred_binary.detach().numpy()

# Create a new DataFrame with the desired columns
predictions_df = pd.DataFrame(data={'Index': X_test.index, 'Prediction': y_pred_numpy})
# Compare the predicted labels with the true labels and count the matches
correct_predictions = (y_pred_binary == y_test_tensor).float().sum().item()

print(f"Number of correctly predicted samples: {correct_predictions}\{y_test_tensor.shape[0]}")

CHAP 4

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer
df = pd.read_csv('./train.csv')
df.describe(include=["object"])
df['input'] = 'TEXT1:'+ df.context + '; TEXT2:'+df.target + '; ANC1: '+ df.anchor
df.input.head()
dataset = Dataset.from_pandas(df)
model_nm = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_nm)
tokenizer.tokenize("I love you. I hate you. I don't know.")
def tok_func(x): return tokenizer(x['input'])

tok_ds = dataset.map(tok_func,batched=True)
row = tok_ds[0]
tok_ds = tok_ds.rename_columns({'score':'labels'})



In [ ]:
eval_df = pd.read_csv('./test.csv')
eval_df.describe()

In [ ]:
from sklearn import datasets

housing : pd.DataFrame  = datasets.fetch_california_housing(as_frame=True) # type: ignore

housing = housing["data"].join(housing["target"]).sample(1000,random_state=52)

housing.head() 


In [ ]:
np.set_printoptions(precision=2,suppress=True)

def corr(x,y): return np.corrcoef(x,y)[0,1]
def show_corr(df,a,b):
    x,y = df[a],df[b]
    plt.scatter(x,y,alpha=0.5,s=4)
    plt.title(f'Correlation between {a} and {b}: {corr(x,y):.2f}')
np.corrcoef(housing,rowvar=True)
corr(housing.MedInc,housing.MedHouseVal)

subset = housing[housing.AveRooms < 15]
show_corr(subset,'MedInc','AveRooms')


In [ ]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

dds = tok_ds.train_test_split(0.25,seed=42)
dds

In [ ]:
from transformers import TrainingArguments,Trainer

bs = 128
epochhs = 4
lr = 8e-5
args = TrainingArguments(
    'ouputs',
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochhs,
    weight_decay=0.01,
    report_to=None,
    learning_rate=lr,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    fp16=True,
    evaluation_strategy="epoch"
)
model = AutoModelForSequenceClassification.from_pretrained(model_nm,num_labels=1)
trainer = Trainer(model,args,train_dataset=dds['train'],eval_dataset=dds['test'],tokenizer=tokenizer,compute_metrics=corr_d) # type: ignore


In [ ]:
trainer.train()

In [ ]:
eval_df['input'] = 'TEXT1:'+ eval_df.context + '; TEXT2:'+eval_df.target + '; ANC1: '+ eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)
preds = trainer.predict(eval_ds) # type: ignore
preds = preds.predictions.astype(float).clip(0,1)     # type: ignore
print(preds)

In [ ]:
import datasets
eval_df['input'] = 'TEXT1:'+ eval_df.context + '; TEXT2:'+eval_df.target + '; ANC1: '+ eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)
preds = trainer.predict(eval_ds) # type: ignore
preds = preds.predictions.astype(float).clip(0,1)     # type: ignore
print(preds)
# assuming preds is a numpy array, remove single-dimensional entries from the shape of the array
simple_preds = preds.squeeze()

# create the submission dataframe
submission = pd.DataFrame({'id': eval_df['id'], 'score': simple_preds})

# save to csv
submission.to_csv('submission.csv',index=False)

OTHER

In [ ]:
path = untar_data(URLs.PETS)
pets = DataBlock(blocks=[ImageBlock, CategoryBlock],
                 get_items=get_image_files,
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=[RandomResizedCrop(460, min_scale=0.75)],
                 batch_tfms=[*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)])

In [ ]:
dls = pets.dataloaders(path/"images")
learn = Learner(dls, resnet18, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.model

In [ ]:
import torch

# Define the input data
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]], dtype=torch.float32)

# Define the target data
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]], dtype=torch.float32)

# Define the model
model = torch.nn.Linear(1, 1)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the model
for epoch in range(1000):
    # Forward pass
    y_pred = model(x)
    loss = loss_fn(y_pred, y)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate the model
y_pred = model(x)
print(y_pred)